In [1]:
import time
import csv
import json
import random
import time
import Levenshtein
import requests
import wikitextparser as wtp
from wiki_dump_reader import iterate
from tqdm import tqdm
from statistics import mean

from extraction_croissement import *

In [2]:
start_loading_data_time = time.time()

# Loading des données sauvegardées dans la mémoire ram
g_book_ADP = rdflib.Graph()
g_author_ADP = rdflib.Graph()
ADP_book_graph = g_book_ADP.parse("../Graphes/grapheADPLivres.rdf")
ADP_author_graph = g_author_ADP.parse("../Graphes/grapheADPAuteurs.rdf")
ADP_books = get_ADP_books(g_book_ADP, g_author_ADP)
ADP_loading_time = time.time()
print("ADP_loading_time: ", ADP_loading_time - start_loading_data_time)

g_item_DL = rdflib.Graph()
book_graph_DL = g_item_DL.parse("../Graphes/grapheDepotLegal.rdf")
DL_books = get_depot_legal_book(g_item_DL)
DL_loading_time = time.time()
print("DL_loading_time: ", DL_loading_time - ADP_loading_time)

# g_item_ILE = rdflib.Graph()
# item_graph_ILE = g_item_ILE.parse("../Graphes/grapheILE.rdf")
# ILE_books = get_ILE_book(g_item_ILE)
# ILE_loading_time = time.time()
# print("ILE_loading time: ", ILE_loading_time - DL_loading_time)

ILE_books = get_ILE_book_csv()
ILE_loading_time = time.time()
print("ILE_loading time: ", ILE_loading_time - DL_loading_time)

books_Hurtubise_file = open("./Hurtubise/Exportation-Hurtubise.csv", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(books_Hurtubise_file, delimiter=',', fieldnames=[
    "Editeur", "ISBN Papier", "ISBN PDF", "ISBN epub", "Titre", "Sous - titre", "Titre de la serie",
    "Contributeurs", "Contributeur(premier)", "Langue", "Langue Origine", "Resume", "Nombre de pages",
    "Date de parution", "Annee de parution", "Sujet  THEMA principal", "Sujet THEMA",
    "Quantificateur Georaphique", "Quantificateur de langue", "Quantificateur Historique", "Niveau soclaire FR",
    "Niveau scolaire QC", "Cycle scolaire FR", "Niveau de lecture", "Echele CECR", "Quantificateur d'interet",
    "Quantificateur d'age", "Quantificateur de style", "Classification Editoriale", "Mots cles"

])
Hurtubise_books = get_Hurtubise_books(csv_reader)
books_Hurtubise_file.close()

authors_ILE_file = open("./ILE/auteurs_ILE_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_ILE_file, delimiter=',', fieldnames=[
    'uri', 'nom', 'bio', 'genres', 'site', 'pseudonyme'])
authors_ILE = [x for x in csv_reader]
authors_ILE_file.close()

authors_wikidata_file = open("./Wikidata/ecrivains_wikidata_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_wikidata_file, delimiter=',', fieldnames=[
    'nom', 'uri'])
authors_wikidata = [x for x in csv_reader]
authors_wikidata_file.close()

authors_DBpedia_file = open("./DBpedia/ecrivains_dbpedia_fr.txt", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_DBpedia_file, delimiter=';', fieldnames=[
    'uri', 'nom'])
authors_DBpedia = [x for x in csv_reader]
authors_DBpedia_file.close()

babelioJsonBooks = open("./Babelio/babelio_livres.json", "r")
Babelio_books = get_Babelio_books(json.load(babelioJsonBooks))
babelioJsonBooks.close()

babelioJsonAuthor = open("./Babelio/babelio_auteurs.json", "r")
Babelio_authors = get_Babelio_books(json.load(babelioJsonAuthor))
babelioJsonAuthor.close()

loading_data_time = time.time()
print("loading_data_time: ", loading_data_time - start_loading_data_time)

all_books = ADP_books + ILE_books + Hurtubise_books + Babelio_books + DL_books
random.shuffle(all_books)

with open('all_books.json', 'w') as outfile:
    json.dump(all_books, outfile)

ADP_loading_time:  10.358738422393799
DL_loading_time:  42.671600580215454
ILE_loading time:  14.520714044570923
loading_data_time:  67.851145029068
recoupement des auteurs
{'total': 21515, 'ADP': 5201, 'ILE': 0, 'Hurtubise': 884, 'Babelio': 1651, 'Depot_legal': 13779}


100%|██████████| 102985/102985 [00:00<00:00, 122249.71it/s]


In [ ]:
with open('all_books.json', 'r') as input_file:
    all_books = json.load(input_file)
author_ls = {}
print('recoupement des auteurs')
for book in tqdm(all_books, total=len(all_books)):
    if len(book['author']) == 1 and isinstance(book['author'][0], list):
        book['author'] = book['author'][0]
    for author in book['author']:
        if author.replace('"', '') not in author_ls:
            author_ls[author.replace('"', '')] = [book]
        else:
            author_ls[author.replace('"', '')].append(book)

with open('authors_ls.json', 'w') as outfile:
    json.dump(author_ls, outfile)


In [3]:
# test sur le croissement des auteurs
famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']
for famous_author in famous:
    for author in author_ls:
        famous_author_part_1, famous_author_part_2 = famous_author.split(" ")
        re_part_1 = re.search(r"{0}".format(famous_author_part_1), author, flags=re.IGNORECASE)
        re_part_2 = re.search(r"{0}".format(famous_author_part_2), author, flags=re.IGNORECASE)
        re_full = re.search(r"{0}".format(famous_author), author, flags=re.IGNORECASE)
        if re_part_1:
            re_part_1_result = re_part_1.group()
        if re_part_2:
            re_part_2_result = re_part_2.group()
        if re_full:
            re_full_result = re_full.group()
        if re_full:
            print("full find: ", famous_author, " with ", author, " in the db")
        elif re_part_1 and re_part_2:
            print("reverse find: ", famous_author, " with ", author, " in the db")
        elif (re_part_1 or re_part_2):
            pass
            #print("partial find: ", famous_author, " with ", author, " in the db")

reverse find:  Michel Tremblay  with  Michel  Tremblay  in the db
full find:  Michel Tremblay  with  Michel Tremblay  in the db
reverse find:  Michel Tremblay  with  Michelle Tremblay-Lacoursière  in the db
full find:  Michel Tremblay  with  Pierre-Michel Tremblay  in the db
reverse find:  Michel Tremblay  with  Michelle  Tremblay Lacoursière  in the db
reverse find:  Michel Tremblay  with  MICHEL G. TREMBLAY  in the db
reverse find:  Anne Hébert  with  Anne  Hébert  in the db
full find:  Anne Hébert  with  Anne Hébert  in the db
full find:  Gabrielle Roy  with  Gabrielle Roy  in the db
reverse find:  Gabrielle Roy  with  Gabrielle  Roy  in the db
full find:  Marie Cardinal  with  Marie Cardinal  in the db
reverse find:  Marie Cardinal  with  Marie-Josée Cardinal  in the db
reverse find:  Réjean Ducharme  with  Réjean  Ducharme  in the db
full find:  Jacques Ferron  with  Jacques Ferron  in the db
reverse find:  Jacques Ferron  with  Jacques  Ferron  in the db
full find:  Victor-Lévy B

In [4]:
"""
Numero de prop importante:
- sex or gender (P21)
- country of citizenship (P27)
- name in native language (P1559)
- birth name (P1477)
-  given name  (P735)
- family name (P734)
- date of birth (P569)
- place of birth (P19)
- date of death  (P570)
- place of death  (P20)
- occupation (P106)
    - author (Q482980)
    - writer (Q36180)
- notable work (P800)
- genre (P136)
- award received (P166)
- nominated for (P1411)
- country of citizenship (P27)

premiere query: ecrivain canadiens parlant/ecrivant en francais => 1232 resultats
SELECT ?item ?itemLabel
WHERE
{{

    ?item wdt:P31 wd:Q5 .
    ?item wdt:P106 wd:Q36180 .
    ?item wdt:P27 wd:Q16 .
    ?item wdt:P1412 wd:Q150

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}}

deuxieme query: ecrivain canadiens parlant/ecrivant en francais qui ont un produit notable => 82 resultats
SELECT ?item ?itemLabel ?bookLabel
WHERE
{

  ?item wdt:P31 wd:Q5 .
  ?item wdt:P106 wd:Q36180 .
  ?item wdt:P27 wd:Q16 .
  ?item wdt:P1412 wd:Q150 .
  ?item wdt:P800 ?book

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
}

"""

q = """
SELECT DISTINCT ?s ?book ?bookLabel
WHERE
{{
    ?s wdt:P31 wd:Q5 .
    {{ ?s wdt:P106  wd:Q36180 }} UNION
    {{ ?s wdt:P106  wd:Q49757 }} UNION
    {{ ?s wdt:P106  wd:Q214917 }} UNION
    {{ ?s wdt:P106  wd:Q381353 }} UNION
    {{ ?s wdt:P106  wd:Q6625963 }} UNION
    {{ ?s wdt:P106  wd:Q4853732 }} UNION
    {{ ?s wdt:P106  wd:Q5434338 }} UNION
    {{ ?s wdt:P106  wd:Q1626130 }} UNION
    {{ ?s wdt:P106  wd:Q10297252 }} UNION
    {{ ?s wdt:P106  wd:Q18844224 }} UNION
    {{ ?s wdt:P106  wd:Q15980158 }} UNION
    {{ ?s wdt:P106  wd:Q26203955 }} UNION
    {{ ?s wdt:P106  wd:Q11774202 }} UNION
    {{ ?s wdt:P106  wd:Q1930187 }} UNION
    {{ ?s wdt:P106  wd:Q487596 }} UNION
    {{ ?s wdt:P106  wd:Q201788 }} .

    {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }} .
    OPTIONAL {{ ?s wdt:P800 ?book . }}

    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr,en".}}
}}
"""

In [12]:
# test with famous authors
res = {}
print('tset query')
count = 0
for author in tqdm(famous, total=len(famous)):
    time.sleep(5)
    test = q.format(author)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))
    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            count += 1
            res[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for resultat in rep_wikidata_ecrivain.json()['results']['bindings']:
                if 'book' in resultat:
                    res[author]['books'].append({'book_id': resultat['book']['value'], 'book_label': resultat['bookLabel']['value']})
print("Nombre d'auteurs trouvés: ", count, " soit ", count * 100/len(famous), "%")

100%|██████████| 11/11 [00:56<00:00,  5.13s/it]


tset query
Nombre d'auteurs trouvés:  9  soit  81.81818181818181 %


In [14]:
res = {}
print('query')
count = 0
# for author in tqdm(author_ls.keys(), total=len(author_ls)):
for author in tqdm(list(author_ls)[:], total=len(list(author_ls)[:])):
    time.sleep(1)
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q.format(author))
    if rep_wikidata_ecrivain.status_code == 429:
        print("trop rapide !!!")
    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            count += 1
            res[author] = {'id':rep_wikidata_ecrivain.json()['results']['bindings'][0]['s']['value'], 'books':[]}
            for resultat in rep_wikidata_ecrivain.json()['results']['bindings']:
                if 'book' in resultat:
                    res[author]['books'].append({'book_id': resultat['book']['value'], 'book_label': resultat['bookLabel']['value']})
print("Nombre d'auteurs trouvés: ", count, " soit ", count * 100/len(author_ls), "%")

with open('wikidata_author_books_list.json', 'w') as outfile:
    json.dump(res, outfile)

100%|██████████| 21515/21515 [7:14:56<00:00,  1.21s/it]   


query
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
trop rapide !!!
Nombre d'auteurs trouvés:  2582  soit  12.000929584011155 %


In [6]:
all_authors_query = """
SELECT DISTINCT ?author ?authorLabel ?bookLabel ?awardLabel
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

    OPTIONAL { ?author wdt:P800 ?book . }
    OPTIONAL { ?author wdt:P166 ?award . }

    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en".}

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)

code_status:  200


In [10]:
with open('Wikidata/request_allfrench_author_wikidata.json') as input_file:
    res = json.load(input_file)

cleaned_data = {}

for result in res:
    if result['author'] not in cleaned_data:
        cleaned_data[result['author']] = {
            'author_uri': result['author'],
            'author_label': result['authorLabel'],
            'works': [result['bookLabel']] if 'bookLable' in result else [],
            'awards': [result['awardLabel']] if 'awardLabel' in result else []
        }
    else:
        if 'bookLabel' in result and result['bookLabel']  not in cleaned_data[result['author']]['works']:
            cleaned_data[result['author']]['works'].append(result['bookLabel'])
        if 'awardLabel' in result and result['awardLabel'] not in cleaned_data[result['author']]['awards']:
            cleaned_data[result['author']]['awards'].append(result['awardLabel'])



# if rep_wikidata_ecrivain.status_code == 200:
#     if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
#         print('exemples: ')
#         print(json.dumps(rep_wikidata_ecrivain.json()['results']['bindings'][:2], indent=2))
#         for result in tqdm(rep_wikidata_ecrivain.json()['results']['bindings']):
#             if result['author']['value'] not in cleaned_data:
#                 cleaned_data[result['author']['value']] = {
#                     'author_uri': result['author']['value'],
#                     'author_label': result['authorLabel']['value'],
#                     'works': [result['bookLabel']['value']] if 'bookLable' in result else [],
#                     'awards': [result['awardLabel']['value']] if 'awardLabel' in result else []
#                 }
#             else:
#                 if 'bookLabel' in result and result['bookLabel']['value']  not in cleaned_data[result['author']['value']]['works']:
#                     cleaned_data[result['author']['value']]['works'].append(result['bookLabel']['value'])
#                 if 'awardLabel' in result and result['awardLabel']['value'] not in cleaned_data[result['author']['value']]['awards']:
#                     cleaned_data[result['author']['value']]['awards'].append(result['awardLabel']['value'])

with open('Wikidata/wikidata_author_books_price_extraction.json', 'w') as output_file:
    json.dump(cleaned_data, output_file)

In [11]:
# Comparaison avec wikidata
with open('Wikidata/wikidata_author_books_price_extraction.json', 'r') as author_file:
    wikidata_authors = json.load(author_file)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file)

found_once = 0
count = 0
count_freq = {}
count_info = 0

couple_writers = []
data_base_stats = {}

for author_DB in tqdm(author_ls, total=len(author_ls)):
    found = False
    count_freq_i = 0
    for author_wikidata_id, author_wikidata in wikidata_authors.items():
        normalised_wikidata_title = normalize(author_wikidata['author_label'])
        dist_auteur = Levenshtein.distance(author_DB, normalised_wikidata_title)
        if max(1, min(len(author_DB), len(normalised_wikidata_title)) / 6) > dist_auteur > max(1, min(len(author_DB), len(normalised_wikidata_title)) / 8):
            print("Autheur DB: ", author_DB + " " * (40 - len(author_DB)), "Autheur wikidata: ", normalised_wikidata_title)
        if dist_auteur < max(1, min(len(author_DB), len(normalised_wikidata_title)) / 8):
            new_couple = {
                        'author DB': author_DB,
                        'author wikidata': author_wikidata['author_label'],
                        'book DB': author_ls[author_DB],
                        'info_wikidata': author_wikidata
                    }
            count += 1
            count_freq_i += 1
            found = True

            couple_writers.append(new_couple)
            for book in author_ls[author_DB]:
                try:
                    data_base_stats[book['data_base']] += 1
                except:
                    data_base_stats[book['data_base']] = 1

print("nombre d'autheurs en commun: ", found_once, " soit ", found_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", count, " soit ", count * 100 / len(list(author_ls)), "%")
print("Nombre d'information catégorisée: ", count_info, " soit ", count_info * 100 / len(list(author_ls)), "%")
print("Frequence de reconnaissance: ", count_freq)
print("repartition dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats))
with open('Wikidata/couple_authors_wikidata.json', 'w') as outfile:
    json.dump(couple_writers, outfile)

100%|██████████| 20059/20059 [59:54<00:00,  5.58it/s] 


Autheur DB:  claude lapierre                          Autheur wikidata:  claude c pierre
Autheur DB:  louise simard                            Autheur wikidata:  louise girard
Autheur DB:  dominique drouin                         Autheur wikidata:  dominique rolin
Autheur DB:  anita mestokosho                         Autheur wikidata:  rita mestokosho
Autheur DB:  pierre bedard                            Autheur wikidata:  pierre abelard
Autheur DB:  bernard couet                            Autheur wikidata:  bernard cohen
Autheur DB:  michel levesque                          Autheur wikidata:  michel j levesque
Autheur DB:  richard petit                            Autheur wikidata:  richard pepin
Autheur DB:  antonio alfonso                          Autheur wikidata:  antonio d alfonso
Autheur DB:  claude renault                           Autheur wikidata:  claude legault
Autheur DB:  jacques benoit                           Autheur wikidata:  jacques w benoit
Autheur DB:  francois la

In [13]:
count_books = 0
count_books_global = []

count_awards = 0
count_awards_global = []

awards_set = set()

for couple in couple_writers:
    if couple['info_wikidata']['works']:
        count_books += 1
        count_books_global.append(len(couple['info_wikidata']['works']))
    if couple['info_wikidata']['awards']:
        count_awards += 1
        count_awards_global.append(len(couple['info_wikidata']['awards']))
        awards_set.update(couple['info_wikidata']['awards'])

print("Nombre d'artiste ayant des livres attribués: ", count_books)
print("Nombre de livres global récupérés: ", sum(count_books_global))

print("Nombre d'artiste ayant des prix attribués: ", count_awards)
print("Nombre de prix global récupérés: ", sum(count_awards_global))

print("exemple des prix: ", awards_set)

Nombre d'artiste ayant des livres attribués:  61
Nombre de livres global récupérés:  193
Nombre d'artiste ayant des prix attribués:  414
Nombre de prix global récupérés:  1117
exemple des prix:  {'Golden Globe de la meilleure actrice dans un film dramatique', "Ours d'or", 'prix Booker', 'Panthéon de la musique canadienne', 'prix Schiller', "prix de poésie de la ville d'Angers", 'prix Mercier', 'prix Sant Jordi du cinéma', 'Prix Gilles-Corbeil', 'Prix de la bande à Mœbius', 'Prix des lecteurs du Marché de la poésie de Montréal', 'Prix du grand public du Salon du livre de Montréal - La Presse', 'PEN/Nabokov Award', 'Prix des Écrivains francophones d’Amérique', 'prix Denise-Pelletier', 'Aurora Award for Best Graphic Novel', 'Gerald Lampert Award', 'prix international Man-Booker', 'prix du scénario du Festival de Cannes', 'prix Tollander', 'Neustadt International Prize for Literature', 'Zamenhof award', 'prix Anthony', 'Stella Prize', 'prix Viareggio', 'Prix littéraire Adrienne-Choquette',

In [32]:
couples_confirmed = []
couples_infirmed = []
couples_unsure = []

count_checked_by_titles = 0
count_no_titles = 0
count_differentiate_by_titles = 0

for couple in tqdm(couple_writers):
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False
    try:
        book_titles_DB = [book['title'] for book in couple['book DB']]
        for title in book_titles_DB:
            normalised_wikidata_title_ls = [normalize(title_wikidata) for title_wikidata in couple['info_wikidata']['works']]
            for title_wikidata in normalised_wikidata_title_ls:
                dist_titles = Levenshtein.distance(title, title_wikidata)
                if dist_titles < max(1, min(len(title), len(title_wikidata)) / 4):
                    confirmed_by_title_match = True
                    break
        if not confirmed_by_title_match:
            for title in book_titles_DB:
                for title_wiki_raw in couple['info_wikidata']['works']:
                    if re.search(normalize(title), normalize(title_wiki_raw)):
                        confirmed_by_title_match_re = True
                        break
    except:
        pass

    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if couple['info_wikidata']['works']:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

# for couple in couple_writers[:100]:
#     print('Auteur DB: ', couple['author DB'], " " * (40 - len(couple['author DB'])),' Auteur wiki ' ,couple['author wiki'], 'confirmé par comparaison de titres: ', couple['confirmed_by_title_match'], 'confirmé par regex: ', couple['confirmed_by_title_match_re'])
print('nombre de couple: ', len(couple_writers))
print('nombre de couples confirmés par titres: ', count_checked_by_titles)
print('nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('Wikidata/couple_authors_wikidata_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile)

with open('Wikidata/couple_authors_wikidata_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile)

with open('Wikidata/couple_authors_wikidata_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile)

100%|██████████| 1357/1357 [00:00<00:00, 17553.61it/s]


nombre de couple:  1357
nombre de couples confirmés par titres:  28
nombre de couples infirmés par titres:  33
nombre de couples sans titres de wikipedia:  1296


In [25]:
count = 0
mean_count = []
totals_price = set()
for couple in couple_writers:
    if couple['info_wikidata']['awards']:
        count += 1
    totals_price.update(couple['info_wikidata']['awards'])
    mean_count.append(len(couple['info_wikidata']['awards']))

print("Nombre d'auteur dont on a récupéré des prix: ", count)
print("Nombre total de prix: ", sum(mean_count))

Nombre d'auteur dont on a récupéré des prix:  414
Nombre total de prix:  1117


In [7]:
# autres informations disponnibles

all_authors_query = """
SELECT DISTINCT ?author
WHERE
{
    ?author wdt:P31 wd:Q5 .
    ?author wdt:P27 ?country .
    ?country wdt:P2936 wd:Q150 .
    ?author wdt:P106 ?occup .
    ?occup wdt:P279 wd:Q36180

}"""
rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query)
print('code_status: ', rep_wikidata_ecrivain.status_code)

authors_uri = []
if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            print('exemples: ')
            print(json.dumps(rep_wikidata_ecrivain.json()['results']['bindings'][:2], indent=2))
            for result in tqdm(rep_wikidata_ecrivain.json()['results']['bindings']):
                authors_uri.append(result['author']['value'])

100%|██████████| 18826/18826 [00:00<00:00, 649352.94it/s]


code_status:  200
exemples: 
[
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q34922"
    }
  },
  {
    "author": {
      "type": "uri",
      "value": "http://www.wikidata.org/entity/Q35109"
    }
  }
]


In [14]:
result = {}
for couple in tqdm(couple_writers):
    uri = couple['info_wikidata']['author_uri']
    time.sleep(1)
    all_authors_query = """
    SELECT ?pLabel ?objLabel
    WHERE {{
        wd:{0} ?predicate ?obj.
        BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?p)
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "fr, en". }}
    }}
    """
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + all_authors_query.format(uri.split('/')[-1]))
    # print('code_status: ', rep_wikidata_ecrivain.status_code)
    if rep_wikidata_ecrivain.status_code == 429:
            print("trop rapide !!!")
    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            result[uri] = [(duo['pLabel']['value'], duo['objLabel']['value']) for duo in rep_wikidata_ecrivain.json()['results']['bindings']]
    else:
        print("erreur code: ", rep_wikidata_ecrivain.status_code)

100%|██████████| 1357/1357 [30:06<00:00,  1.33s/it]


In [18]:
prop_dict = {}
prop_counter = {}
for author_uri, properties in result.items():
    for property in properties:
        try:
            prop_counter[property[0]] += 1
        except:
            prop_counter[property[0]] = 1

for author_uri, properties in result.items():
    for rep_prop in prop_counter:
        counter = 0
        for property in properties:
            if property[0] == rep_prop:
                counter += 1
        try:
            prop_dict[rep_prop].append(counter)
        except:
            prop_dict[rep_prop] = [counter]

prop_resume = {}
for rep_prop in prop_dict:
    prop_resume[rep_prop] = {
        'non_null_mean': mean([e for e in prop_dict[rep_prop] if e]),
        'nb_non_null': sum([1 for e in prop_dict[rep_prop] if e]),
        'sum': sum(prop_dict[rep_prop]),
    }

# print(json.dumps(sorted(prop_resume.items(), key=lambda item: item[1]['nb_non_null'], reverse=True), indent=1))
with open('Wikidata/list_properies_by_author.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'non_null_mean', 'nb_non_null', 'sum' ])
        writer.writeheader()
        for rep_prop_name, rep_prop_dict in prop_resume.items():
            if rep_prop_name.find("http://www.wikidata.org/prop/") == -1:
                rep_prop_dict['name'] = rep_prop_name
                writer.writerow(rep_prop_dict)

